In [10]:
from flask import Flask, render_template, request, json, jsonify

import numpy as np
import torch 
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer
from transformers import BertForSequenceClassification

import tweepy
import time
import emoji

import matplotlib.pyplot as plt
import pandas as pd
#import plotly.express as px
from datetime import datetime, timedelta

import base64

app = Flask(__name__)
app.config['JSON_AS_ASCII'] = False

@app.route("/index", methods=['GET', 'POST'])
def index():  
    if request.method == 'GET':
        return render_template('index.html')
    if request.method == 'POST':
        query = request.form.get('userInput')
        if request.form.get('func_select') == "keyword":
            h = get_tweet_text(query)
            get_tweet_user_keyword(h)
            get_tweet_sentiment_keyword()
            get_query_count(query)
            return render_template('serchWord.html', data=tweet_text_sentiment)
        
        elif request.form.get('func_select') == "id":
            data_tuple = find_relationship(query)
            tweet_id_7_day = get_user_sentiment_7_day()
            uae = get_user_all_engagement(tweet_id_7_day)
            return render_template('test.html', data=users_name_url)
        
        elif request.form.get('func_select') == "reply":
            get_tweet(query)
            get_reply(query)
            get_tweet_sentiment_reply()
            get_tweet_user_reply()
            return render_template('analyzeReply.html', data=tweet_reply_text_sentiment)    

#config
API_KEY = 'oVXQ6vLK9d8IPdvrBHmhZfDjf'
API_SECRET = 'xu7931ppKzwQYZgu3RTqOkpucCJhefwwXoS9Gpqc9DiJAakNB6'
BEARER_TOKEN = 'AAAAAAAAAAAAAAAAAAAAAG6yUQEAAAAAx6dUsYJnS4yGXdHen9LiEezMl7k%3D5TrrwU9Zl2N0eWKLbb5vGr8LuEariTYxNlul5aANnCaO4UQVZv'
ACCESS_TOKEN = '1397285769790722049-3ZIa8lwlFDe0tiDiESDSzXj4esEmID'
ACCESS_TOKEN_SECRET = 'gVDNVOUrbV55RUUu5x75xF8Ed5xbI3es3jo8S0f6NYzVx'

client = tweepy.Client(bearer_token = BEARER_TOKEN)

users_name_url = {"twitter":[]}
tweet_text_sentiment = {'twitter':[]}
tweet_reply_text_sentiment = {'twitter':[]}
tweet_id_7_day = []
users_list = []


model = torch.load('model3.pth',map_location ='cpu')
token = BertTokenizer.from_pretrained('bert-base-chinese')

def find_relationship(id):
    ids = [id]
    #get user_id
    users = client.get_users(usernames=id, user_fields = ['profile_image_url'])
    user_main = {}
    for user in users.data:
        USER_ID = user.id
        USER_NAME = user.name
        USER_URL = user.profile_image_url
        USER_USERNAME = user.username

    #get user tweet_id
    tweet_id = []
    t = client.get_users_tweets(id = USER_ID, max_results = 50, tweet_fields = ['conversation_id', 'in_reply_to_user_id'])
    for i in t.data:
        tweet_id.append(i.id)

    #get tweets reply_user
    reply_user = []
    for k in range(len(tweet_id)):
        query = 'conversation_id:' + str(tweet_id[k])
        l = client.search_recent_tweets(query = query, max_results = 50, expansions = ['author_id'])
        try:
            for i in l.data:
                reply_user.append(i.author_id)
        except:
            print("", end = "")
    
    #get tweets liking_users
    liking_users = []
    for k in range(len(tweet_id)):
        try:
            l = client.get_liking_users(id = tweet_id[k])
        except:
            print("sleep60")
            time.sleep(60)
        try:
            for i in l.data:
                liking_users.append(i.id)
        except:
            print("", end = "")
    
    #get tweets retweeters
    retweeters = []
    for k in range(len(tweet_id)):
        try:
            l = client.get_retweeters(id = tweet_id[k])
        except:
            print("sleep60")
            time.sleep(60)
        try:
            for i in l.data:
                retweeters.append(i.id)
        except:
            print("", end = "")
    

    #count relationship
    total = {}
    for i in liking_users:
        if i in total.keys():
            total[i] += 2
        else:
            total[i] = 2

    for i in retweeters:
        if i in total.keys():
            total[i] += 2
        else:
            total[i] = 2

    for i in reply_user:
        if i in total.keys():
            total[i] += 3
        else:
            total[i] = 3
    
    total.pop(USER_ID, 87)
    print(total)
    
    result = []
    
    if len(total.items()) >= 60:
        result = sorted(total.items(), key=lambda x:x[1])[-60:]
        users_name_url['people'] = 61
    else:
        result = sorted(total.items(), key=lambda x:x[1])[:]
        users_name_url['people'] = len(total.items()) + 1
    
    result_rev = []
    for i in range(len(result)-1, -1, -1):
        result_rev.append(result[i])
    
    #top 10 user_data
    related_id = {}
    user_name = {}
    for i in result:
        u = client.get_users(ids = i[0], user_fields = ['profile_image_url'])
        for k in u.data:
            related_id[k.id] = []
            user_name[k.id] = k.name
            users_name_url["twitter"].append({"id":str(k.id), "name":k.name, "username":k.username, "url":k.profile_image_url})
    users_name_url["twitter"].append({"id":str(USER_ID), "name":USER_NAME, "username":USER_USERNAME, "url":USER_URL})
    for i in range(len(users_name_url['twitter'])):
        users_name_url['twitter'][i]['url'] = users_name_url['twitter'][i]['url'].replace("normal.jpg", "400x400.jpg")
    return users_name_url



def get_user_sentiment_7_day():
    dtformat = '%Y-%m-%dT%H:%M:%SZ'
    times = datetime.now()
    start_time = times - timedelta(days=6)
    start_time, end_time = start_time.strftime(dtformat), time.strftime(dtformat)

    tweets_id_7days = []

    for k in range(len(users_name_url['twitter'])):
        tmp = []
        tweets_7days = []
        idss = int(users_name_url['twitter'][k]['id'])
        for response in tweepy.Paginator(client.get_users_tweets, id = idss, max_results=100, start_time = start_time, end_time = end_time, tweet_fields = ["referenced_tweets", "created_at"], limit=10):
            for i in response:
                tmp.append(i)
        if tmp[0] != None:
            for i in range(0, len(tmp), 4):
                tmpp = []
                try:
                    for j in range(len(tmp[i])):
                        tweets_7days.append({"id":users_name_url['twitter'][k]['id'], "text":tmp[i][j].text, "time":tmp[i][j].created_at})
                        if(k == len(users_name_url['twitter'])-1):
                            tweets_id_7days.append(tmp[i][j].id)
                except:
                    print("")
                    
            for i in range(len(tweets_7days)):
                tweets_7days[i]["text"] = denoise_text(tweets_7days[i]["text"])
                text = tweets_7days[i]["text"]
                """try:
                    tweets_7days[i]["text"] = translator.translate(text, dest='zh-tw').text
                except:
                    print(end = "")"""

            for j in range(len(tweets_7days)):
                test_dataset = Dataset([tweets_7days[j]["text"]], [-1])
                testloader = DataLoader(dataset=test_dataset, batch_size=1, collate_fn=collate_fn)
                for i in testloader:
                    tokens_tensors, segments_tensors, masks_tensors = i[:3]
                    outputs = model(input_ids=tokens_tensors, 
                        token_type_ids=segments_tensors, 
                        attention_mask=masks_tensors)
                    logits = outputs[0]
                    labels = i[3]
                    _, pred = torch.max(logits.data, 1)
                    if pred[0] == 0:
                        tweets_7days[j]["sentiment"] = 0
                    elif pred[0] == 1:
                        tweets_7days[j]["sentiment"] = 1
                    elif pred[0] == 2:
                        tweets_7days[j]["sentiment"] = 2
                    elif pred[0] == 3:
                        tweets_7days[j]["sentiment"] = 3
            
            sentiment_0 = {}
            sentiment_1 = {}
            sentiment_2 = {}
            sentiment_3 = {}
            plt_day = []
            plt_day_2 = []
            
            for i in range(7):
                tt = times - timedelta(days=i)
                sentiment_0[tt.day] = 0
                sentiment_1[tt.day] = 0
                sentiment_2[tt.day] = 0
                sentiment_3[tt.day] = 0
                plt_day.append(tt.day)
                plt_day_2.append(str(tt.month) + "/" + str(tt.day))

            for i in range(len(tweets_7days)):
                if tweets_7days[i]['sentiment'] == 0:
                        sentiment_0[tweets_7days[i]['time'].day] += 1
                elif tweets_7days[i]['sentiment'] == 1:
                        sentiment_1[tweets_7days[i]['time'].day] += 1
                elif tweets_7days[i]['sentiment'] == 2:
                        sentiment_2[tweets_7days[i]['time'].day] += 1
                elif tweets_7days[i]['sentiment'] == 3:
                        sentiment_3[tweets_7days[i]['time'].day] += 1

            plt_sentiment_0 = []
            plt_sentiment_1 = []
            plt_sentiment_2 = []
            plt_sentiment_3 = []
            
            plt_day.reverse()
            plt_day_2.reverse()
            
            for i in plt_day:
                plt_sentiment_0.append(sentiment_0[i])
                plt_sentiment_1.append(sentiment_1[i])
                plt_sentiment_2.append(sentiment_2[i])
                plt_sentiment_3.append(sentiment_3[i])
            
            
            
            users_name_url['twitter'][k]['data'] = [plt_sentiment_0, plt_sentiment_1, plt_sentiment_2, plt_sentiment_3, plt_day_2]
            
            sentiment = [0,0,0,0]

            for i in range(len(tweets_7days)):
                if tweets_7days[i]['sentiment'] == 0:
                        sentiment[0] += 1
                elif tweets_7days[i]['sentiment'] == 1:
                        sentiment[1] += 1
                elif tweets_7days[i]['sentiment'] == 2:
                        sentiment[2] += 1
                elif tweets_7days[i]['sentiment'] == 3:
                        sentiment[3] += 1
                        
            m = max(sentiment)
            for u in range(4):
                if m == sentiment[u]:
                    if u == 0:
                        users_name_url['twitter'][k]['sentiment'] = "happy"
                    if u == 1:
                        users_name_url['twitter'][k]['sentiment'] = "angry"
                    if u == 2:
                        users_name_url['twitter'][k]['sentiment'] = "sorrow"
                    if u == 3:
                        users_name_url['twitter'][k]['sentiment'] = "normal"
                        
    for k in range(len(users_name_url['twitter'])):
        if 'data' not in users_name_url['twitter'][k]:
            users_name_url['twitter'][k]['data'] = [[0,0,0,0,0,0,0], [0,0,0,0,0,0,0], [0,0,0,0,0,0,0], [0,0,0,0,0,0,0], plt_day_2]
        if 'sentiment' not in users_name_url['twitter'][k]:
            users_name_url['twitter'][k]['sentiment'] = "normal"
    return tweets_id_7days

def get_user_all_engagement(tweets_id_7days):
    reply_user = []
    for k in range(len(tweets_id_7days)):
        query = 'conversation_id:' + str(tweets_id_7days[k])
        l = client.search_recent_tweets(query=query, max_results = 100, expansions = ['author_id'])
        try:
            for i in l.data:
                reply_user.append(i.author_id)
        except:
            print("", end = "")

    liking_users = []
    for k in range(len(tweets_id_7days)):
        try:
            l = client.get_liking_users(id = tweets_id_7days[k])
        except:
            print("sleep360")
            time.sleep(360)
        try:
            for i in l.data:
                liking_users.append(i.id)
        except:
            print("", end = "")

    retweeters = []
    for k in range(len(tweets_id_7days)):
        try:   
            l = client.get_retweeters(id = tweets_id_7days[k])
        except:
            print("sleep360")
            time.sleep(360)
        try:
            for i in l.data:
                retweeters.append(i.id)
        except:
            print("", end = "")


    user_all_Engagement = {"twitter":[]}
    a = []
    for i in range(len(users_name_url['twitter'])):
        a.append(int(users_name_url['twitter'][i]['id']))
        user_all_Engagement['twitter'].append({"id":users_name_url['twitter'][i]['id']})

    for i in liking_users:
        if i in a:
            for k in range(len(user_all_Engagement['twitter'])):
                if int(user_all_Engagement['twitter'][k]['id']) == i:
                    if "liking" not in user_all_Engagement['twitter'][k].keys():
                        user_all_Engagement['twitter'][k]['liking'] = 0
                    elif "liking" in user_all_Engagement['twitter'][k].keys():
                        user_all_Engagement['twitter'][k]['liking'] += 1

    for i in retweeters:
        if i in a:
            for k in range(len(user_all_Engagement['twitter'])):
                if int(user_all_Engagement['twitter'][k]['id']) == i:
                    if "retweet" not in user_all_Engagement['twitter'][k].keys():
                        user_all_Engagement['twitter'][k]['retweet'] = 0
                    elif "retweet" in user_all_Engagement['twitter'][k].keys():
                        user_all_Engagement['twitter'][k]['retweet'] += 1
    for i in reply_user:
        if i in a:
            for k in range(len(user_all_Engagement['twitter'])):
                if int(user_all_Engagement['twitter'][k]['id']) == i:
                    if "reply" not in user_all_Engagement['twitter'][k].keys():
                        user_all_Engagement['twitter'][k]['reply'] = 0
                    elif "reply" in user_all_Engagement['twitter'][k].keys():
                        user_all_Engagement['twitter'][k]['reply'] += 1
    
    for i in range(len(user_all_Engagement['twitter'])):
        if "liking" not in user_all_Engagement['twitter'][i].keys():
            user_all_Engagement['twitter'][i]['liking'] = 0
        if "retweet" not in user_all_Engagement['twitter'][i].keys():
            user_all_Engagement['twitter'][i]['retweet'] = 0
        if "reply" not in user_all_Engagement['twitter'][i].keys():
            user_all_Engagement['twitter'][i]['reply'] = 0
    for i in range(len(user_all_Engagement['twitter'])):
        users_name_url['twitter'][i]['liking'] = user_all_Engagement['twitter'][i]['liking']
        users_name_url['twitter'][i]['retweet'] = user_all_Engagement['twitter'][i]['retweet']
        users_name_url['twitter'][i]['reply'] = user_all_Engagement['twitter'][i]['reply']

    return user_all_Engagement

def get_tweet_text(query):
    query = query
    k = 0
    user_list = []
    tweet_list = []
    tweet_text_sentiment['keyword'] = query
    for response in tweepy.Paginator(client.search_recent_tweets, query = query, tweet_fields = ['referenced_tweets'], expansions = ["referenced_tweets.id", 'author_id'], max_results = 100, limit = 30):
        for tweet in response.data:
            if k == 1000:
                tweet_text_sentiment['count'] = len(tweet_list)
                return user_list
            if tweet.id not in tweet_list and tweet.referenced_tweets != None and tweet.referenced_tweets[0]['type'] != "retweeted":
                tmp = denoise_text(eval(repr(tweet.text)))
                """try:
                        tmp = translator.translate(tmp, dest='zh-tw').text
                    except:
                        print(end = "")"""
                user_list.append(tweet.author_id)
                tweet_text_sentiment['twitter'].append({'id':str(tweet.id), 'text':tmp})
                tweet_list.append(tweet.id)
                k+=1
        for tweet in response.includes['tweets']:
            if k == 1000:
                tweet_text_sentiment['count'] = len(tweet_list)
                return user_list
            if tweet.id not in tweet_list:
                tmp = denoise_text(eval(repr(tweet.text)))
                """try:
                        tmp = translator.translate(tmp, dest='zh-tw').text
                    except:
                        print(end = "")"""
                user_list.append(tweet.author_id)
                tweet_text_sentiment['twitter'].append({'id':str(tweet.id), 'text':tmp})
                tweet_list.append(tweet.id)
                k+=1
    tweet_text_sentiment['count'] = len(tweet_list)
    return user_list

def get_tweet_user_keyword(user_list):
    tmp = []
    k = 0
    m = 0
    for j in range(len(user_list)):
        tmp.append(user_list[j])
        k+=1
        if (k == 100) or (j == len(user_list) - 1):
            users = client.get_users(ids = tmp)
            for user in users.data:
                tweet_text_sentiment['twitter'][m]['username'] = denoise_text(user.username)
                tweet_text_sentiment['twitter'][m]['name'] = denoise_text(user.name)
                m += 1
            k = 0
            tmp = []

def get_query_count(query):
    times = datetime.now()
    counts = client.get_recent_tweets_count(query=query, granularity='day')
    day_count = []
    tmp = []
    for count in counts.data:
        tmp.append(count['tweet_count'])
    bb = []
    for i in range(7):
        tt = times - timedelta(days=i+1)
        bb.append(str(tt.month) + "/" + str(tt.day))
    day_count.append(tmp)
    bb.reverse()
    day_count.append(bb)
    tweet_text_sentiment['line_data'] = day_count
    
def get_tweet_sentiment_keyword():
    sentiment = [0,0,0,0]
    for j in range(len(tweet_text_sentiment['twitter'])):
        test_dataset = Dataset([tweet_text_sentiment['twitter'][j]['text']], [-1])
        testloader = DataLoader(dataset=test_dataset, batch_size=1, collate_fn=collate_fn)
        for i in testloader:
            tokens_tensors, segments_tensors, masks_tensors = i[:3]
            outputs = model(input_ids=tokens_tensors, 
                token_type_ids=segments_tensors, 
                attention_mask=masks_tensors)
            logits = outputs[0]
            labels = i[3]
            _, pred = torch.max(logits.data, 1)
            if pred[0] == 0:
                tweet_text_sentiment['twitter'][j]['semtiment'] = "happy"
                sentiment[0] += 1
            elif pred[0] == 1:
                tweet_text_sentiment['twitter'][j]['semtiment'] = "angry"
                sentiment[1] += 1
            elif pred[0] == 2:
                tweet_text_sentiment['twitter'][j]['semtiment'] = "sorrow"
                sentiment[2] += 1
            elif pred[0] == 3:
                tweet_text_sentiment['twitter'][j]['semtiment'] = "normal"
                sentiment[3] += 1
    tweet_text_sentiment['data'] = sentiment 

def get_tweet(query):
    response = client.get_tweet(id = query, expansions = ['author_id'])
    tweet_reply_text_sentiment['twitter'].append({"text":denoise_text(response.data.text)})
    users_list.append(response.data.author_id)

def get_reply(query):
    q = 'conversation_id:' + str(query)
    count = 0
    for response in tweepy.Paginator(client.search_recent_tweets, q, max_results=100, expansions = ['author_id'], tweet_fields = ['conversation_id', 'in_reply_to_user_id','referenced_tweets'], limit=30):
        for data in response.data:
            tweet_reply_text_sentiment['twitter'].append({"text":denoise_text(data.text)})
            users_list.append(data.author_id)
            count += 1
    tweet_reply_text_sentiment['count'] = count

def get_tweet_user_reply():
    tmp = []
    k = 0
    m = 0
    for j in range(len(users_list)):
        tmp.append(users_list[j])
        k+=1
        if (k == 100) or (j == len(users_list) - 1):
            users = client.get_users(ids = tmp)
            for user in users.data:
                tweet_reply_text_sentiment['twitter'][m]['username'] = denoise_text(user.username)
                tweet_reply_text_sentiment['twitter'][m]['name'] = denoise_text(user.name)
                m += 1
            k = 0
            tmp = []
    
def get_tweet_sentiment_reply():
    sentiment = [0,0,0,0]
    for j in range(len(tweet_reply_text_sentiment['twitter'])):
        test_dataset = Dataset([tweet_reply_text_sentiment['twitter'][j]['text']], [-1])
        testloader = DataLoader(dataset=test_dataset, batch_size=1, collate_fn=collate_fn)
        for i in testloader:
            tokens_tensors, segments_tensors, masks_tensors = i[:3]
            outputs = model(input_ids=tokens_tensors, 
                token_type_ids=segments_tensors, 
                attention_mask=masks_tensors)
            logits = outputs[0]
            labels = i[3]
            _, pred = torch.max(logits.data, 1)
            if pred[0] == 0:
                tweet_reply_text_sentiment['twitter'][j]['semtiment'] = "happy"
                sentiment[0] += 1
            elif pred[0] == 1:
                tweet_reply_text_sentiment['twitter'][j]['semtiment'] = "angry"
                sentiment[1] += 1
            elif pred[0] == 2:
                tweet_reply_text_sentiment['twitter'][j]['semtiment'] = "sorrow"
                sentiment[2] += 1
            elif pred[0] == 3:
                tweet_reply_text_sentiment['twitter'][j]['semtiment'] = "normal"
                sentiment[3] += 1
    tweet_reply_text_sentiment['data'] = sentiment    


class Dataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, i):
        return self.x[i], self.y[i]

def collate_fn(data):
    sents = [data[i][0] for i in range(len(data))]
    labels = [data[i][1] for i in range(len(data))]
    
    data = token.batch_encode_plus(batch_text_or_text_pairs=sents,
                    truncation=True, 
                    padding='max_length',
                    max_length=50,
                    return_tensors='pt',
                    return_length=True)
    
    input_ids = data['input_ids'] 
    attention_mask = data['attention_mask'] 
    token_type_ids = data['token_type_ids'] 
    labels = torch.LongTensor(labels) 
    
    return input_ids, attention_mask, token_type_ids, labels

def get_predictions(model, dataloader, compute_acc=False):
    predictions = None
    correct = 0
    total = 0
    tmp = []
    with torch.no_grad():
        for data in dataloader:
            
            tokens_tensors, segments_tensors, masks_tensors = data[:3]
            outputs = model(input_ids=tokens_tensors, 
                            token_type_ids=segments_tensors, 
                            attention_mask=masks_tensors)
            
            logits = outputs[0]
            _, pred = torch.max(logits.data, 1)
            tmp.append(pred[0])
    return tmp

import re
from bs4 import BeautifulSoup

from googletrans import Translator
translator = Translator()

#Removeing @...
def remove_pattern(text):
    return re.sub('@[\w]*', '', text)

#Removeing http
def remove_http(text):
    results = re.compile(r'[http|https]*://[a-zA-Z0-9._?/&=:]*', re.S) 
    return re.sub(results, '', text)

#Removing the html strips
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removeing 
def remove_emoji(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

#Removing the noisy text
def denoise_text(text):
    text = remove_pattern(text)
    text = remove_http(text)
    text = remove_emoji(text)
    text = text.replace('\n', '')
    text = text.replace('\\', '')
    text = text.replace('\'', '`')
    text = text.replace('\"', '`')
    text = text.replace('\u3000', '')
    text = text.replace('\xa0', '')
    text = emoji.demojize(text)
    return text

class Dataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, i):
        return self.x[i], self.y[i]

def collate_fn(data):
    sents = [data[i][0] for i in range(len(data))]
    labels = [data[i][1] for i in range(len(data))]
    
    data = token.batch_encode_plus(batch_text_or_text_pairs=sents,
                    truncation=True, 
                    padding='max_length',
                    max_length=50,
                    return_tensors='pt',
                    return_length=True)
    
    input_ids = data['input_ids'] 
    attention_mask = data['attention_mask'] 
    token_type_ids = data['token_type_ids'] 
    labels = torch.LongTensor(labels) 
    
    return input_ids, attention_mask, token_type_ids, labels

def get_predictions(model, dataloader, compute_acc=False):
    predictions = None
    correct = 0
    total = 0
    tmp = []
    with torch.no_grad():
        for data in dataloader:
            
            tokens_tensors, segments_tensors, masks_tensors = data[:3]
            outputs = model(input_ids=tokens_tensors, 
                            token_type_ids=segments_tensors, 
                            attention_mask=masks_tensors)
            
            logits = outputs[0]
            _, pred = torch.max(logits.data, 1)
            tmp.append(pred[0])
    return tmp


def return_img_stream(img_local_path):
    img_stream = ''
    with open(img_local_path, 'rb') as img_f:
        img_stream = img_f.read()
        img_stream = base64.b64encode(img_stream).decode()
    return img_stream


In [11]:
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [03/Dec/2022 11:56:55] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [03/Dec/2022 11:56:56] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [03/Dec/2022 11:57:02] "GET /index HTTP/1.1" 200 -
127.0.0.1 - - [03/Dec/2022 11:57:02] "GET /static/index_setting.css HTTP/1.1" 200 -
127.0.0.1 - - [03/Dec/2022 11:57:18] "POST /index HTTP/1.1" 200 -
127.0.0.1 - - [03/Dec/2022 11:57:18] "GET /static/keyword_setting.css HTTP/1.1" 404 -
127.0.0.1 - - [03/Dec/2022 11:57:19] "GET /index HTTP/1.1" 200 -
127.0.0.1 - - [03/Dec/2022 11:59:33] "POST /index HTTP/1.1" 200 -
127.0.0.1 - - [03/Dec/2022 11:59:33] "GET /static/reply_setting.css HTTP/1.1" 404 -
127.0.0.1 - - [03/Dec/2022 11:59:34] "GET /index HTTP/1.1" 200 -
[2022-12-03 12:00:02,387] ERROR in app: Exception on /index [POST]
Traceback (most recent call last):
  File "C:\Users\MEA\anaconda3\lib\site-packages\flask\app.py", line 2446, in wsgi_app
    response = self.full_dispatch_reque

{1227409542079598593: 2, 1311695063953539073: 4, 1251159908818419712: 58, 722116272373108736: 2, 1494739545371250690: 2, 1131400803556548609: 2, 996339886234918912: 6, 1545307923722645504: 2, 2719213189: 2, 387501070: 2, 1102350282304811008: 2, 1083614870874910720: 2, 701983740759658496: 8, 471737381: 2, 3067418742: 2, 230987476: 2, 1520846709714132992: 2, 383722226: 6, 1037747281548664833: 4, 1272866310267912193: 2, 1217573304141111296: 2, 2507494303: 2, 1101801411250012161: 2, 1424554886348247044: 4, 1384470337547423753: 10, 3046874840: 2, 877985307802468352: 2, 1556215532109824000: 2, 2278545222: 2, 1670373938: 2, 700708076802826241: 6, 1327237803021795328: 4, 1509539178274467843: 6, 1394899180733095937: 2, 1085317037713371136: 4, 597464399: 2, 978032881: 2, 971734625767706625: 2, 1591744608555651073: 8, 1419103160656822276: 2, 1305441085875867649: 4, 2324906599: 2, 1463436251856474120: 2, 1598863569730572288: 8, 1535899389645860864: 2, 949905027493081089: 4, 1026809300096667648: 2,

127.0.0.1 - - [03/Dec/2022 12:06:14] "POST /index HTTP/1.1" 200 -
127.0.0.1 - - [03/Dec/2022 12:06:14] "GET /static/detailPage_arrange.css HTTP/1.1" 404 -
127.0.0.1 - - [03/Dec/2022 12:06:14] "GET /static/concertric.css HTTP/1.1" 200 -
127.0.0.1 - - [03/Dec/2022 12:06:35] "GET /index HTTP/1.1" 200 -


In [2]:
import json

def get_relation_json_print(user_id):
    start = time.time()
    a = find_relationship(user_id)
    b = get_user_sentiment_7_day()
    c = get_user_all_engagement(b)
    j = json.dumps(users_name_url, ensure_ascii=False)
    end = time.time()
    print("執行時間：%f 秒" % (end - start))
    print(j)

def get_keyword_json_print(query):
    start = time.time()
    h = get_tweet_text(query)
    get_tweet_user_keyword(h)
    get_tweet_sentiment_keyword()
    get_query_count(query)
    j = json.dumps(tweet_text_sentiment, ensure_ascii=False)
    end = time.time()
    print("執行時間：%f 秒" % (end - start))
    print(j)
    
def get_reply_json_print(query):
    start = time.time()
    get_tweet(query)
    get_reply(query)
    get_tweet_sentiment_reply()
    get_tweet_user_reply()
    j = json.dumps(tweet_reply_text_sentiment, ensure_ascii=False)
    end = time.time()
    print("執行時間：%f 秒" % (end - start))
    print(j)

In [10]:
get_keyword_json_print("孤獨搖滾")

執行時間：16.868971 秒
{"twitter": [{"id": "1598005497537646593", "text": " 我控制不了我自己⋯救命⋯孤獨搖滾對我做了什麼⋯", "username": "181f19", "name": "歹", "semtiment": "angry"}, {"id": "1597981712851795969", "text": "謝謝槍搜面和孤獨搖滾讓我有動漫分類 ", "username": "tokkibear__", "name": "토끼베어:teddy_bear::white_heart:", "semtiment": "happy"}, {"id": "1597962999347965955", "text": "孤獨搖滾整部作品最愛的就是PA醬 好想要被她壓在地上用厭惡的眼神看著 ", "username": "JKSASHIMI", "name": "SASHIMI", "semtiment": "happy"}, {"id": "1597918935894364162", "text": " 我現在只想看下一集孤獨搖滾.jpg", "username": "SSR__TL", "name": "Trifling Light －SSR", "semtiment": "sorrow"}, {"id": "1597882797246410752", "text": " 孤獨搖滾還真是本季黑馬，但鏈鋸人我是不行", "username": "brahmantw", "name": "ब्रह्मन्", "semtiment": "angry"}, {"id": "1597877935431888903", "text": " 孤獨搖滾超好看", "username": "bud_222", "name": "芽呱", "semtiment": "happy"}, {"id": "1597822348086116352", "text": "帥帥ㄉ不良啾菜孤獨搖滾加油啊!吉他英雄!!50音胡梨老師~~ ", "username": "sSGxpQZIkGwACX3", "name": "可以吃ㄉ水豚 (オニテンジクネズミ", "semtiment": "happy"}, {"id": "15978151

In [5]:
get_reply_json_print(1597822944709074944)

執行時間：5.752295 秒
{"twitter": [{"text": "來片披薩? ", "semtiment": "normal", "username": "Sibylart", "name": "SIBYLα 西貝魯阿法"}, {"text": " 太香了 我的口水:drooling_face::drooling_face:", "semtiment": "happy", "username": "Iris97046229", "name": "Lycoris"}, {"text": " 太美啦 如此可愛的披薩哪裡可以買的到 謝謝老師", "semtiment": "happy", "username": "rubyruby9420", "name": "石英龍"}, {"text": " 這片披薩5星好平(被打", "semtiment": "normal", "username": "uK64nY2kpTF19pt", "name": "極光Aurora"}, {"text": " 沒有 看過 這麼 可愛的 披薩:rolling_on_the_floor_laughing:", "semtiment": "happy", "username": "Aguang0814", "name": "阿光"}, {"text": " 感謝西貝魯老師畫的美美大小姐(*˘︶˘*).。.:*♡", "semtiment": "happy", "username": "IjyuUtOM8BGb2dE", "name": "五華晴香"}, {"text": " 些些我死了", "semtiment": "angry", "username": "ayakanzaki_aya", "name": "神崎あやKanzaki Aya"}, {"text": " 這披薩太可愛了！:heart_suit:(⁠≧⁠▽⁠≦⁠):heart_suit:感謝西貝魯老師~(⁠*⁠´⁠ω⁠｀⁠*⁠)", "semtiment": "happy", "username": "bTiDyZ98OQu4XeV", "name": "烈鋒"}, {"text": "  4b老師RIP", "semtiment": "happy", "username": "sechan_732426", "name

In [3]:
a = get_relation_json_print("durayray")

{1162940310470590464: 2, 860551388505309184: 4, 1494237570431356929: 2, 1085896504265428992: 2, 804264122: 2, 1515745379861819393: 2, 919403254005473280: 2, 1178890128216842241: 2, 1293800998821957632: 2, 999292238046814209: 6, 1171098835067604992: 2, 1449627477744054272: 6, 4661886252: 2, 1147907593202913281: 2, 1169304841492951040: 2, 2313832256: 2, 1273588978013229059: 4, 1069876145309413376: 2, 2829752072: 2, 1125281622: 4, 2579766350: 2, 2203053373: 8, 1439937208350171136: 6, 973754624330014720: 4, 316638731: 7, 1147476647873925120: 2, 1413102479693148161: 2, 1519209400249962497: 2, 833256449329344512: 2, 870924511331229696: 2, 339569431: 6, 1551203349545922561: 4, 1598530320625594368: 6, 1528289918484566018: 2, 1472837394588377088: 4, 937258433140613120: 2, 1465356565876457472: 28, 903252124145819648: 2, 1487379077476794368: 2, 2579065118: 2, 3842827274: 2, 1235893143750340609: 2, 2433479695: 4, 1006569907079692288: 2, 904994203884593153: 2, 715370090: 2, 1429469438311501830: 2, 

執行時間：206.343976 秒
{"twitter": [{"id": "919117668212285440", "name": "灰塵", "username": "dusttt_39", "url": "https://pbs.twimg.com/profile_images/1521792060390797312/Q6XLIugp_400x400.jpg", "data": [[0, 2, 5, 1, 4, 3, 0], [0, 0, 0, 0, 1, 0, 0], [0, 0, 1, 0, 1, 1, 0], [0, 1, 0, 0, 0, 0, 0], ["11/27", "11/28", "11/29", "11/30", "12/1", "12/2", "12/3"]], "sentiment": "happy", "liking": 3, "retweet": 0, "reply": 0}, {"id": "1230918755531649025", "name": "popper_z", "username": "Renoir30678", "url": "https://pbs.twimg.com/profile_images/1562405118439464961/5lyNUgRc_400x400.jpg", "data": [[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], ["11/27", "11/28", "11/29", "11/30", "12/1", "12/2", "12/3"]], "sentiment": "normal", "liking": 1, "retweet": 0, "reply": 0}, {"id": "1381190208394158084", "name": "楊榮聖", "username": "SamChia78017377", "url": "https://pbs.twimg.com/profile_images/1381192095902273546/TF8i7sJZ_400x400.jpg", "data": [[0, 0, 0, 0, 0, 0, 0],

In [4]:
users_name_url

{'twitter': [{'id': '919117668212285440',
   'name': '灰塵',
   'username': 'dusttt_39',
   'url': 'https://pbs.twimg.com/profile_images/1521792060390797312/Q6XLIugp_400x400.jpg',
   'data': [[0, 2, 5, 1, 4, 3, 0],
    [0, 0, 0, 0, 1, 0, 0],
    [0, 0, 1, 0, 1, 1, 0],
    [0, 1, 0, 0, 0, 0, 0],
    ['11/27', '11/28', '11/29', '11/30', '12/1', '12/2', '12/3']],
   'sentiment': 'happy',
   'liking': 3,
   'retweet': 0,
   'reply': 0},
  {'id': '1230918755531649025',
   'name': 'popper_z',
   'username': 'Renoir30678',
   'url': 'https://pbs.twimg.com/profile_images/1562405118439464961/5lyNUgRc_400x400.jpg',
   'data': [[0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0],
    ['11/27', '11/28', '11/29', '11/30', '12/1', '12/2', '12/3']],
   'sentiment': 'normal',
   'liking': 1,
   'retweet': 0,
   'reply': 0},
  {'id': '1381190208394158084',
   'name': '楊榮聖',
   'username': 'SamChia78017377',
   'url': 'https://pbs.twimg.com/profile_images

In [6]:
tweet_reply_text_sentiment

{'twitter': [{'text': '來片披薩? ',
   'semtiment': 'normal',
   'username': 'Sibylart',
   'name': 'SIBYLα 西貝魯阿法'},
  {'text': ' 太香了 我的口水:drooling_face::drooling_face:',
   'semtiment': 'happy',
   'username': 'Iris97046229',
   'name': 'Lycoris'},
  {'text': ' 太美啦 如此可愛的披薩哪裡可以買的到 謝謝老師',
   'semtiment': 'happy',
   'username': 'rubyruby9420',
   'name': '石英龍'},
  {'text': ' 這片披薩5星好平(被打',
   'semtiment': 'normal',
   'username': 'uK64nY2kpTF19pt',
   'name': '極光Aurora'},
  {'text': ' 沒有 看過 這麼 可愛的 披薩:rolling_on_the_floor_laughing:',
   'semtiment': 'happy',
   'username': 'Aguang0814',
   'name': '阿光'},
  {'text': ' 感謝西貝魯老師畫的美美大小姐(*˘︶˘*).。.:*♡',
   'semtiment': 'happy',
   'username': 'IjyuUtOM8BGb2dE',
   'name': '五華晴香'},
  {'text': ' 些些我死了',
   'semtiment': 'angry',
   'username': 'ayakanzaki_aya',
   'name': '神崎あやKanzaki Aya'},
  {'text': ' 這披薩太可愛了！:heart_suit:(\u2060≧\u2060▽\u2060≦\u2060):heart_suit:感謝西貝魯老師~(\u2060*\u2060´\u2060ω\u2060｀\u2060*\u2060)',
   'semtiment': 'happy',
   'userna